In [3]:
import os
os.environ['OPENAI_API_KEY']=open('./../../keys.txt','r').read()
os.environ['SERPAPI_API_KEY']=open('./../../keys2.txt','r').read()
os.environ['GOOGLE_CSE_ID']=open('./../../keys3.txt','r').read()
os.environ['GOOGLE_API_KEY']=open('./../../keys4.txt','r').read()
os.environ['WOLFRAM_ALPHA_APPID'] = open('./../../keys5.txt','r').read()

In [4]:
from langchain.document_loaders import YoutubeLoader
from langchain.llms import OpenAI
from langchain.chains.summarize import load_summarize_chain

In [ ]:
# pip install youtube-transcript-api
# pip install pytube

### 1. Simple Videos

In [8]:
loader = YoutubeLoader.from_youtube_channel(youtube_url="https://www.youtube.com/watch?v=i8NETqtGHms",add_video_info=True)
# from_youtube_url("https://www.youtube.com/watch?v=g2o22C3CRfU", add_video_info=True)

In [9]:
result = loader.load()

In [11]:
print (type(result))
# print (f"Found video from {result[0].metadata['author']} that is {result[0].metadata['length']} seconds long")
print ("")
print (result)

<class 'list'>

[Document(page_content="tensorflow an open source machine learning framework famous for powering deep neural networks with high-level code it was developed by the google brain team and first released in 2015. it's most commonly used with python but can run in other languages like javascript c plus plus and java at its core it's just a library for programming with linear algebra and statistics as you know the word tensor describes a multilinear relationship between sets of algebraic objects within a vector space aka a multi-dimensional array what makes it special is its collection of apis for data processing visualization model evaluation and deployment that make deep learning accessible to the average developer it's extremely portable and is able to run on tiny mobile cpus or microcontrollers with tensorflow lite can run in the browser with tensorflow.js while the core library can scale up to multiple gpus or run on tensor processing units ships engineered specifically 

In [12]:
llm = OpenAI(temperature=0)

In [13]:
chain = load_summarize_chain(llm, chain_type="stuff", verbose=False)
chain.run(result)

' TensorFlow is an open source machine learning framework developed by the Google Brain Team in 2015. It is most commonly used with Python, but can also run in other languages. It is used for data processing, visualization, model evaluation, and deployment, and is portable and able to run on tiny mobile CPUs or microcontrollers. It is used in many applications, such as medicine, Twitter, Spotify, and PayPal. To build a neural network, one must create a Python file, install TensorFlow, and use the Keras library to build the network layer by layer. The end result is a model that can make predictions with the likelihood that an image is a certain type of clothing.'

### 2. Long Videos

In [15]:
loader = YoutubeLoader.from_youtube_channel(youtube_url="https://www.youtube.com/watch?v=k9J4H5gkLqw")
# ("https://www.youtube.com/watch?v=k9J4H5gkLqw", add_video_info=True)

In [16]:
result = loader.load()

In [17]:
chain = load_summarize_chain(llm, chain_type="stuff", verbose=False)
chain.run(result)

InvalidRequestError: This model's maximum context length is 4097 tokens, however you requested 9505 tokens (9249 in your prompt; 256 for the completion). Please reduce your prompt; or completion length.

Problem, your transcript/document is too long

In [18]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [19]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=0)

In [20]:
texts = text_splitter.split_documents(result)

In [23]:
len(texts)

20

In [24]:
chain = load_summarize_chain(llm, chain_type="map_reduce", verbose=True)
chain.run(texts)



> Entering new MapReduceDocumentsChain chain...
Prompt after formatting:
Write a concise summary of the following:


"- Everybody keeps saying, "Drew, you can't restock your merch. You don't know how. You are too stupid." But guess what, loser? I figured it out. And now you can buy more
shirts at drewgoodenshop.com. Hey guy, welcome back to another episode of my braless wife bringing me
a sandwich I didn't ask for. So I made a Twitch account
in 2018 with the intention of streaming regularly. And in the four years
sense, I have streamed once. I keep telling myself, I'll go back to it but as with everything else in life, I remember it's just
so much easier not to. So I don't. But today, that all changes. Thanks to the help of one man. Tyler "Ninja" Blevins. If you don't know who Ninja is, he's sort of the Michael
Jordan of having blue hair. If you do know who Ninja is, you probably expected me to
say he's the Michael Jordan of Twitch streaming. But that would imply that he's
one of the

' Tyler "Ninja" Blevins is a popular Twitch streamer who has released a masterclass on streaming. The class covers topics such as equipment, branding, and interacting with viewers. It also mentions Ninja\'s blue hair, which he believes is integral to his success. The speaker also talks about their own experiences with streaming, such as buying a better chair and changing their background. They also mention HelloFresh and offer a promo code for up to 16 free meals and three surprise gifts.'

### 3. Multiple Videos

In [26]:
youtube_url_list = ["https://www.youtube.com/watch?v=7sQCmgCmN7U", "https://www.youtube.com/watch?v=5JBMvUMJuyQ"]

texts = []

text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=0)

for url in youtube_url_list:
    # loader = YoutubeLoader.from_youtube_url(url, add_video_info=True)
    loader = YoutubeLoader.from_youtube_channel(youtube_url=url,add_video_info=True)
    result = loader.load()
    
    texts.extend(text_splitter.split_documents(result))

In [29]:
chain = load_summarize_chain(llm, chain_type="map_reduce", verbose=False)
chain.run(texts)

" Tsar's Cove is a family of coronaviruses that can cause Ovid 19, a contagious viral infection that affects the throat and lungs. Medical device companies are ramping up production of ventilators to help those with severe symptoms of the virus, and researchers are working to develop a vaccine. This article also looks at the story of James Bartley who was allegedly swallowed by a whale in 1891 and examines the potential effects of the whale's stomach acids on a human body. It is debunked that Bartley survived, as the person would be forced down the throat, unable to breathe due to lack of oxygen and increase in methane gas, and hydrochloric acid would eat away at the skin."